### Section I: Prerequisites

#### 1.0. Import Required Libraries

In [0]:
import os
import json
import pymongo
import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

#### 2.0. Instantiate Global Variables

In [0]:
# Azure MySQL Server Connection Information ###################
jdbc_hostname = "yvf7ua-mysql.mysql.database.azure.com"
jdbc_port = 3306
src_database = "sakila_dw"

connection_properties = {
  "user" : "yvf7ua",
  "password" : "Passw0rd123",
  "driver" : "org.mariadb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "clustermidterm.7rtqy6x"
atlas_database_name = "sakila_dw"
atlas_user_name = "yvf7ua"
atlas_password = "Passw0rd123"

# Data Files (JSON) Information ###############################
dst_database = "sakila_dlh"

base_dir = "dbfs:/FileStore/ds2002-project2"
database_dir = f"{base_dir}/{dst_database}"

data_dir = f"{base_dir}/source_data"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"

casting_stream_dir = f"{stream_dir}"

casting_output_bronze = f"{database_dir}/fact_casting/bronze"
casting_output_silver = f"{database_dir}/fact_casting/silver"
casting_output_gold   = f"{database_dir}/fact_casting/gold"


# Delete the Streaming Files ################################## 
dbutils.fs.rm(f"{database_dir}/fact_casting", True) 

# Delete the Database Files ###################################
dbutils.fs.rm(database_dir, True)

Out[8]: False

#### 3.0. Define Global Functions

In [0]:
# ######################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
# ######################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

# ######################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
# ######################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

### Section II: Populate Dimensions by Ingesting Reference (Cold-path) Data 
#### 1.0. Fetch Reference Data From an Azure MySQL Database
##### 1.1. Create a New Databricks Metadata Database.

In [0]:
%sql
DROP DATABASE IF EXISTS sakila_dlh CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS sakila_dlh
COMMENT "DS-2002 Project 2"
LOCATION "dbfs:/FileStore/ds2002-project2/sakila_dlh"
WITH DBPROPERTIES (contains_pii = true, purpose = "DS-2002 Project2");

##### 1.2. Create a New Table that Sources Date Dimension Data from an Azure MySQL database.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://yvf7ua-mysql.mysql.database.azure.com:3306/sakila_dw",
  dbtable "dim_date",
  user "yvf7ua",
  password "Passw0rd123"
)

In [0]:
%sql
USE DATABASE sakila_dlh;

CREATE OR REPLACE TABLE sakila_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/ds2002-lab06/northwind_dlh/dim_date"
AS SELECT * FROM view_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_date;

col_name,data_type,comment
date_key,int,null
full_date,date,null
date_name,string,null
date_name_us,string,null
date_name_eu,string,null
day_of_week,int,null
day_name_of_week,string,null
day_of_month,int,null
day_of_year,int,null
weekday_weekend,string,null


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_date LIMIT 5

date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,week_of_year,month_name,month_of_year,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


##### 1.3. Create a New Table that Sources Product Dimension Data from an Azure MySQL database.

In [0]:
%sql
-- Create a Temporary View named "view_actor" that extracts data from your MySQL Northwind database.
CREATE OR REPLACE TEMPORARY VIEW view_actor
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://yvf7ua-mysql.mysql.database.azure.com:3306/sakila_dw",
  dbtable "dim_actor",
  user "yvf7ua",
  password "Passw0rd123"
)

In [0]:
%sql
USE DATABASE sakila_dlh;
-- Create a new table named "northwind_dlh.dim_product" using data from the view named "view_product"
CREATE OR REPLACE TABLE sakila_dlh.dim_actor
COMMENT "Product Dimension Table"
LOCATION "dbfs:/FileStore/ds2002-lab06/northwind_dlh/dim_actor"
AS SELECT * FROM view_actor

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_actor;

col_name,data_type,comment
actor_key,bigint,null
first_name,string,null
last_name,string,null
last_update,timestamp,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,sakila_dlh,
Table,dim_actor,
Type,EXTERNAL,


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_actor LIMIT 5

actor_key,first_name,last_name,last_update
1,PENELOPE,GUINESS,2006-02-15T04:34:33.000+0000
2,NICK,WAHLBERG,2006-02-15T04:34:33.000+0000
3,ED,CHASE,2006-02-15T04:34:33.000+0000
4,JENNIFER,DAVIS,2006-02-15T04:34:33.000+0000
5,JOHNNY,LOLLOBRIGIDA,2006-02-15T04:34:33.000+0000


#### 2.0. Fetch Reference Data from a MongoDB Atlas Database
##### 2.1. View the Data Files on the Databricks File System

In [0]:
display(dbutils.fs.ls(batch_dir))

path,name,size,modificationTime
dbfs:/FileStore/ds2002-project2/source_data/batch/dim_actor.json,dim_actor.json,7842,1683666410000
dbfs:/FileStore/ds2002-project2/source_data/batch/dim_date.json,dim_date.json,138125,1683666410000
dbfs:/FileStore/ds2002-project2/source_data/batch/dim_film.json,dim_film.json,136548,1683666410000
dbfs:/FileStore/ds2002-project2/source_data/batch/dim_film_actor.json,dim_film_actor.json,28698,1683666410000


##### 2.2. Create a New MongoDB Database, and Load JSON Data Into a New MongoDB Collection
**NOTE:** The following cell **can** be run more than once because the **set_mongo_collection()** function **is** idempotent.

In [0]:
source_dir = '/dbfs/FileStore/ds2002-project2/source_data/batch'
json_files = {"film" : 'dim_film.json'}

set_mongo_collection(atlas_user_name, atlas_password, atlas_cluster_name, atlas_database_name, source_dir, json_files) 

Out[29]: <pymongo.results.InsertManyResult at 0x7f0681177b40>

##### 2.3.1. Fetch Film Dimension Data from the New MongoDB Collection

In [0]:
%scala
import com.mongodb.spark._

val df_film = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "sakila_dw").option("collection", "film").load()
.select("film_key","title","description","release_year","length","replacement_cost","rating")

display(df_film)


film_key,title,description,release_year,length,replacement_cost,rating
1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,86,20.99,PG
2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,48,12.99,G
3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,50,18.99,NC-17
4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank,2006,117,26.99,G
5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,2006,130,22.99,G
6,AGENT TRUMAN,A Intrepid Panorama of a Robot And a Boy who must Escape a Sumo Wrestler in Ancient China,2006,169,17.99,PG
7,AIRPLANE SIERRA,A Touching Saga of a Hunter And a Butler who must Discover a Butler in A Jet Boat,2006,62,28.99,PG-13
8,AIRPORT POLLOCK,A Epic Tale of a Moose And a Girl who must Confront a Monkey in Ancient India,2006,54,15.99,R
9,ALABAMA DEVIL,A Thoughtful Panorama of a Database Administrator And a Mad Scientist who must Outgun a Mad Scientist in A Jet Boat,2006,114,21.99,PG-13
10,ALADDIN CALENDAR,A Action-Packed Tale of a Man And a Lumberjack who must Reach a Feminist in Ancient China,2006,63,24.99,NC-17


In [0]:
%scala
df_film.printSchema()

root
-- film_key: integer (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- release_year: integer (nullable = true)
-- length: integer (nullable = true)
-- replacement_cost: double (nullable = true)
-- rating: string (nullable = true)

##### 2.3.2. Use the Spark DataFrame to Create a New Customer Dimension Table in the Databricks Metadata Database (northwind_dlh)

In [0]:
%scala
df_film.write.format("delta").mode("overwrite").saveAsTable("sakila_dlh.dim_film")

In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_film

col_name,data_type,comment
film_key,int,null
title,string,null
description,string,null
release_year,int,null
length,int,null
replacement_cost,double,null
rating,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_film LIMIT 5

film_key,title,description,release_year,length,replacement_cost,rating
1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,86,20.99,PG
2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,48,12.99,G
3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,50,18.99,NC-17
4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank,2006,117,26.99,G
5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,2006,130,22.99,G


#### 3.0. Fetch Data from a File System
##### 3.1. Use PySpark to Read From a CSV File

In [0]:
film_actor_csv = f"{batch_dir}/dim_film_actor.csv"

df_film_actor = spark.read.format('csv').options(header='true', inferSchema='true').load(film_actor_csv)
display(df_film_actor)

actor_key,film_key,last_update
1,1,2006-02-15T05:05:03.000+0000
1,23,2006-02-15T05:05:03.000+0000
1,25,2006-02-15T05:05:03.000+0000
1,106,2006-02-15T05:05:03.000+0000
1,140,2006-02-15T05:05:03.000+0000
1,166,2006-02-15T05:05:03.000+0000
1,277,2006-02-15T05:05:03.000+0000
1,361,2006-02-15T05:05:03.000+0000
1,438,2006-02-15T05:05:03.000+0000
1,499,2006-02-15T05:05:03.000+0000


In [0]:
df_film_actor.printSchema()

root
 |-- actor_key: integer (nullable = true)
 |-- film_key: integer (nullable = true)
 |-- last_update: timestamp (nullable = true)



In [0]:
df_film_actor.write.format("delta").mode("overwrite").saveAsTable("sakila_dlh.dim_film_actor")

In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_film_actor;

col_name,data_type,comment
actor_key,int,null
film_key,int,null
last_update,timestamp,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,sakila_dlh,
Table,dim_film_actor,
Type,MANAGED,
Location,dbfs:/FileStore/ds2002-project2/sakila_dlh/dim_film_actor,


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_film_actor limit 5;

actor_key,film_key,last_update
1,1,2006-02-15T05:05:03.000+0000
1,23,2006-02-15T05:05:03.000+0000
1,25,2006-02-15T05:05:03.000+0000
1,106,2006-02-15T05:05:03.000+0000
1,140,2006-02-15T05:05:03.000+0000


##### Verify Dimension Tables

In [0]:
%sql
USE sakila_dlh;
SHOW TABLES

database,tableName,isTemporary
sakila_dlh,dim_actor,false
sakila_dlh,dim_date,false
sakila_dlh,dim_film,false
sakila_dlh,dim_film_actor,false
,view_actor,true
,view_date,true


### Section III: Integrate Reference Data with Real-Time Data
#### 6.0. Use AutoLoader to Process Streaming (Hot Path) Orders Fact Data 
##### 6.1. Bronze Table: Process 'Raw' JSON Data

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 .option("cloudFiles.schemaHints", "actor_key BIGINT")
 .option("cloudFiles.schemaHints", "first_name STRING")
 .option("cloudFiles.schemaHints", "last_name STRING")
 .option("cloudFiles.schemaHints", "film_key BIGINT")
 .option("cloudFiles.schemaHints", "title STRING")
 .option("cloudFiles.schemaHints", "description STRING")
 .option("cloudFiles.schemaHints", "release_year BIGINT")
 .option("cloudFiles.schemaHints", "length BIGINT")
 .option("cloudFiles.schemaHints", "rating STRING")
 .option("cloudFiles.schemaHints", "last_update BIGINT")
 .option("cloudFiles.schemaLocation", casting_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(casting_stream_dir)
 .createOrReplaceTempView("casting_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW casting_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM casting_raw_tempview
)

In [0]:
%sql
SELECT * FROM casting_bronze_tempview

actor_key,description,fact_casting_key,film_key,first_name,last_name,last_update_key,length,rating,release_year,title,last_update,_rescued_data,receipt_time,source_file
1,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,1,1,PENELOPE,GUINESS,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
10,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2,1,CHRISTIAN,GABLE,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
20,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,3,1,LUCILLE,TRACY,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
30,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,4,1,SANDRA,PECK,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
40,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,5,1,JOHNNY,CAGE,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
53,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,6,1,MENA,TEMPLE,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
108,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,7,1,WARREN,NOLTE,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
162,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,8,1,OPRAH,KILMER,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
188,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,9,1,ROCK,DUKAKIS,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json
198,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,10,1,MARY,KEITEL,20060215,86,PG,2006,ACADEMY DINOSAUR,null,null,2023-05-10T02:47:50.839+0000,dbfs:/FileStore/ds2002-project2/source_data/stream/casting/fact_casting.json


In [0]:
(spark.table("casting_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{casting_output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_casting_bronze"))

Out[52]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0664d711f0>

##### 6.2. Silver Table: Include Reference Data

In [0]:
(spark.readStream
  .table("fact_casting_bronze")
  .createOrReplaceTempView("casting_silver_tempview"))

In [0]:
%sql
SELECT * FROM casting_silver_tempview

actor_key,description,fact_casting_key,film_key,first_name,last_name,last_update_key,length,rating,release_year,title,last_update,_rescued_data,receipt_time,source_file


In [0]:
%sql
DESCRIBE EXTENDED casting_silver_tempview

col_name,data_type,comment
actor_key,bigint,null
description,string,null
fact_casting_key,bigint,null
film_key,bigint,null
first_name,string,null
last_name,string,null
last_update_key,bigint,null
length,bigint,null
rating,string,null
release_year,bigint,null


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_casting_silver_tempview AS (
  SELECT o.actor_key,
      o.first_name,
      o.last_name,
      o.film_key,
      o.title,
      o.description,
      o.release_year,
      o.length,
      o.rating,
      od.day_name_of_week AS order_day_name_of_week,
      od.day_of_month AS order_day_of_month,
      od.weekday_weekend AS order_weekday_weekend,
      od.month_name AS order_month_name,
      od.calendar_quarter AS order_quarter,
      od.calendar_year AS order_year
  FROM casting_silver_tempview AS o
  LEFT OUTER JOIN sakila_dlh.dim_date AS od
  ON od.date_key = o.last_update
)

In [0]:
(spark.table("fact_casting_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{casting_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_casting_silver"))

Out[58]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0664c929d0>

In [0]:
%sql
SELECT * FROM fact_casting_silver

actor_key,first_name,last_name,film_key,title,description,release_year,length,rating,order_day_name_of_week,order_day_of_month,order_weekday_weekend,order_month_name,order_quarter,order_year


In [0]:
%sql
DESCRIBE EXTENDED fact_casting_silver

col_name,data_type,comment
actor_key,bigint,null
first_name,string,null
last_name,string,null
film_key,bigint,null
title,string,null
description,string,null
release_year,bigint,null
length,bigint,null
rating,string,null
order_day_name_of_week,string,null


##### 6.3. Gold Table: Perform Aggregations

In [0]:

%sql
SELECT last_name, 
        COUNT(*) as number_of_films 
    FROM sakila_dlh.fact_casting_silver
    GROUP BY last_name
    ORDER BY number_of_films DESC

last_name,number_of_films


In [0]:
%sql
-- Author a query that returns the Total Quantity grouped by the Quarter Created, Inventory Transaction Type, and Product
-- Sort by the Total Quantity Descending
SELECT quantity AS Total_Quantity
  , order_quarter AS Quarter_Created
  , inventory_transaction_type AS Inventory_Transaction_Type
  , product_name AS Product
FROM northwind_dlh.fact_inventory_transactions_silver
GROUP BY Product, Quarter_Created, Inventory_Transaction_Type, Total_Quantity
ORDER BY Total_Quantity DESC

#### Clean up the File System

In [0]:
%fs rm -r /FileStore/ds2002-lab06/